<a href="https://colab.research.google.com/github/CateLe/gcp/blob/master/GCP_BigQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Before you begin


1.   Use the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) to Create a Cloud Platform project if you do not already have one.
2.   [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
3.   [Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) APIs for the project.


### Provide your credentials to the runtime

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## Optional: Enable data table display

Colab includes the ``google.colab.data_table`` package that can be used to display large pandas dataframes as an interactive data table.
It can be enabled with:

In [ ]:
%load_ext google.colab.data_table

If you would prefer to return to the classic Pandas dataframe display, you can disable this by running:
```python
%unload_ext google.colab.data_table
```

# Use BigQuery via magics

The `google.cloud.bigquery` library also includes a magic command which runs a query and either displays the result or saves it to a variable as a `DataFrame`.

In [ ]:
# Display query output immediately

%%bigquery --project silent-album-362714
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

,total_rows
0,114420316


In [ ]:
# Save output in a variable `df`

%%bigquery --project silent-album-362714 df
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

In [ ]:
df.shape

(1, 1)

In [ ]:
total_rows = df.iloc[0,0]

# Use BigQuery through google-cloud-bigquery

See [BigQuery documentation](https://cloud.google.com/bigquery/docs) and [library reference documentation](https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/usage.html).

The [GSOD sample table](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=samples&t=gsod&page=table) contains weather information collected by NOAA, such as precipitation amounts and wind speeds from late 1929 to early 2010.


### Declare the Cloud project ID which will be used throughout this notebook

In [ ]:
project_id = 'silent-album-362714'

### Sample approximately 2000 random rows

In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

sample_count = 2000
row_count = client.query('''
  SELECT 
    COUNT(*) as total
  FROM `bigquery-public-data.samples.gsod`''').to_dataframe().total[0]

df = client.query('''
  SELECT
    *
  FROM
    `bigquery-public-data.samples.gsod`
  WHERE RAND() < %d/%d
''' % (sample_count, row_count)).to_dataframe()

print('Full dataset has %d rows' % row_count)

Full dataset has 114420316 rows


### Describe the sampled data

In [ ]:
df.describe()

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,...,num_mean_station_pressure_samples,mean_visibility,num_mean_visibility_samples,mean_wind_speed,num_mean_wind_speed_samples,max_sustained_wind_speed,max_gust_wind_speed,max_temperature,total_precipitation,snow_depth
count,1957.000000,1957.000000,1957.000000,1957.000000,1957.000000,1957.000000,1957.000000,1865.000000,1865.000000,1507.000000,...,702.000000,1750.000000,1750.000000,1935.000000,1935.000000,1903.000000,254.000000,1956.000000,1794.000000,127.000000
mean,507230.404701,89680.429739,1986.905979,6.478283,15.750639,52.153756,12.992846,41.341233,12.929223,1015.205308,...,12.249288,12.198286,12.592571,6.677468,12.961240,11.949921,24.080315,43.443456,0.078590,12.470866
std,296722.783065,27131.553129,16.558484,3.475177,8.697199,24.187448,7.863609,22.531626,7.872531,9.671392,...,7.884090,8.741510,7.756828,4.895050,7.832597,6.553590,9.508220,24.222195,0.417284,12.426509
min,10010.000000,91.000000,1934.000000,1.000000,1.000000,-49.299999,4.000000,-48.799999,4.000000,975.799988,...,4.000000,0.000000,4.000000,0.000000,4.000000,1.000000,6.000000,-55.799999,0.000000,0.400000
25%,265120.000000,99999.000000,1977.000000,3.000000,9.000000,38.400002,7.000000,28.799999,7.000000,1009.500000,...,6.000000,6.425000,7.000000,3.400000,7.000000,7.800000,17.950000,31.525000,0.000000,3.100000
50%,517090.000000,99999.000000,1989.000000,6.000000,16.000000,55.500000,8.000000,44.200001,8.000000,1015.200012,...,8.000000,9.900000,8.000000,5.700000,8.000000,10.900000,22.000000,47.099998,0.000000,9.100000
75%,725047.000000,99999.000000,2001.000000,10.000000,23.000000,70.400002,23.000000,57.500000,23.000000,1021.200012,...,23.000000,14.900000,23.000000,8.750000,23.000000,15.000000,28.900000,60.799999,0.010000,16.700000
max,999999.000000,99999.000000,2010.000000,12.000000,31.000000,102.000000,24.000000,80.699997,24.000000,1064.099976,...,24.000000,99.400002,24.000000,50.599998,24.000000,62.000000,84.900002,93.000000,10.040000,70.900002


### View the first 10 rows

In [ ]:
df.head(10)

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,...,min_temperature,min_temperature_explicit,total_precipitation,snow_depth,fog,rain,snow,hail,thunder,tornado
0,126950,99999,1954,11,9,53.200001,4,44.700001,4.0,1007.500000,...,None,None,NaN,NaN,False,False,False,False,False,False
1,949999,91,1965,9,16,55.799999,4,35.200001,4.0,NaN,...,None,None,0.00,NaN,False,False,False,False,False,False
2,984440,99999,1973,3,10,80.199997,8,70.599998,7.0,1014.400024,...,None,None,0.00,NaN,False,False,False,False,False,False
3,104270,99999,1976,8,19,59.000000,24,47.900002,24.0,NaN,...,None,None,0.00,NaN,False,False,False,False,False,False
4,729680,99999,1977,1,15,-24.299999,8,-34.400002,8.0,1029.000000,...,None,None,0.00,35.799999,True,True,True,True,True,True
5,787620,99999,1982,7,3,77.800003,8,62.400002,8.0,NaN,...,None,None,0.00,NaN,False,False,False,False,False,False
6,332750,99999,1986,7,7,75.599998,8,59.200001,7.0,1010.400024,...,None,None,0.00,NaN,False,False,False,False,False,False
7,282160,99999,1989,7,15,74.199997,8,63.799999,8.0,1014.200012,...,None,None,0.08,NaN,False,False,False,False,False,False
8,722725,99999,1989,11,27,54.700001,14,28.200001,14.0,NaN,...,None,None,0.00,NaN,False,False,False,False,False,False
9,113820,99999,1997,9,17,47.299999,12,36.700001,12.0,1023.700012,...,None,None,0.00,NaN,False,False,False,False,False,False


In [ ]:
# 10 highest total_precipitation samples
df.sort_values('total_precipitation', ascending=False).head(10)[['station_number', 'year', 'month', 'day', 'total_precipitation']]

,station_number,year,month,day,total_precipitation
1590,150900,1983,8,11,10.04
1353,478170,1954,6,29,8.35
1087,305370,1965,8,11,5.91
1015,722067,1980,3,11,4.06
389,586060,1975,2,20,2.76
1544,85510,2009,12,6,2.68
1513,946090,2001,7,30,2.32
273,728740,1973,5,11,2.13
764,295760,1973,9,15,1.85
1773,917650,1993,2,19,1.85


# Use BigQuery through pandas-gbq

The `pandas-gbq` library is a community led project by the pandas community. It covers basic functionality, such as writing a DataFrame to BigQuery and running a query, but as a third-party library it may not handle all BigQuery features or use cases.

[Pandas GBQ Documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_gbq.html)

In [ ]:
import pandas as pd

sample_count = 2000
df = pd.io.gbq.read_gbq('''
  SELECT name, SUM(number) as count
  FROM `bigquery-public-data.usa_names.usa_1910_2013`
  WHERE state = 'TX'
  GROUP BY name
  ORDER BY count DESC
  LIMIT 100
''', project_id=project_id, dialect='standard')

df.head()

,name,count
0,James,272793
1,John,235139
2,Michael,225320
3,Robert,220399
4,David,219028


# Syntax highlighting
`google.colab.syntax` can be used to add syntax highlighting to any Python string literals which are used in a query later.

In [ ]:
from google.colab import syntax
query = syntax.sql('''
SELECT
  COUNT(*) as total_rows
FROM
  `bigquery-public-data.samples.gsod`
''')

pd.io.gbq.read_gbq(query, project_id=project_id, dialect='standard')

,total_rows
0,114420316
